##### Import depenedencies

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_community.document_loaders import PyPDFLoader

import psycopg2
from pgvector.psycopg2 import register_vector

##### PSQL Connection

In [7]:
try:
    conn = psycopg2.connect(os.getenv("POSTGRES_URI"))
    register_vector(conn)
    cursor = conn.cursor()

    # Simple ping
    cursor.execute("SELECT 1;")
    result = cursor.fetchone()

    print("✅ PostgreSQL connection successful:", result)

except Exception as e:
    print("❌ PostgreSQL connection failed:")
    print(e)

✅ PostgreSQL connection successful: (1,)


##### PDF Ingestion pipeline

In [12]:
def ingest_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    return docs


In [16]:
docs = ingest_pdf("Swapnil_Resume_Nov.pdf")
docs

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-03-26T10:09:11+05:30', 'author': 'Austin, Heather', 'moddate': '2025-03-26T10:09:11+05:30', 'source': 'Swapnil_Resume_Nov.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Swapnil Katiyar  \n+91-7503533658     •     swapnil240695@gmail.com    •     Noida, India \nFront-End Developer \nNext.js    |    React.js    |    JavaScript (ES6+)    |    Git   |    Jira \nPassionate Front-End Developer skilled in HTML, CSS, JavaScript, React, and Next.js, with hands-on experience in \nbuilding responsive and interactive web applications. Strong focus on delivering high-quality, maintainable code and \ncrafting exceptional user experiences through collaboration and innovation. \nPROFESSIONAL SKILLS \n• Frontend Development: HTML5, CSS3, JavaScript, TypeScript, React, Next.js \n• Frameworks/Libraries: Redux, Material-UI, React Hook Form, Tailwind CSS \n• Version Control

##### Clean Documents

In [27]:
import re


def clean_text(text):
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r"Page \d+", "", text)
    return text.strip()

for doc in docs:
    doc.page_content = clean_text(doc.page_content)
    print(doc.page_content)

Swapnil Katiyar  
+91-7503533658     •     swapnil240695@gmail.com    •     Noida, India 
Front-End Developer 
Next.js    |    React.js    |    JavaScript (ES6+)    |    Git   |    Jira 
Passionate Front-End Developer skilled in HTML, CSS, JavaScript, React, and Next.js, with hands-on experience in 
building responsive and interactive web applications. Strong focus on delivering high-quality, maintainable code and 
crafting exceptional user experiences through collaboration and innovation. 
PROFESSIONAL SKILLS 
• Frontend Development: HTML5, CSS3, JavaScript, TypeScript, React, Next.js 
• Frameworks/Libraries: Redux, Material-UI, React Hook Form, Tailwind CSS 
• Version Control & Collaboration: Git, Jira 
• Testing: React Testing Library, Jest, Vitest 
WORK EXPERIENCE 
Treeroot Informatics – Ahmedabad, Gujarat, India January 2023 – April 2024 
Front End Developer 
• Worked closely with a team of 8 developers to deliver high-quality front-end features for projects, utilizing 
React and 